In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

In [ ]:
def fitted_svm(params):
    svm = SVC(**params)
    fitted_svm = svm.fit(X_train, Y_train)
    return fitted_svm

def ClassifierMetrics (X_train, Y_train, X_test, Y_test, fitted_model):
    Y_pred = fitted_model.predict(X_test)
    metrics = {'train_accuracy': fitted_model.score(X_train, Y_train)
    'test_accuracy': fitted_model.score(X_test, Y_test)
    'test_auc': roc_auc_score(Y_test, Y_pred)
    'test_ap': average_precision_score(Y_test, Y_pred)}
    return metrics

In [ ]:
svm_model_params = {'kernel':[],'C':[], 'train_accuracy':[],'test_accuracy':[],'test_auc':[], 'test_r2':[]}

In [ ]:
svm_pd = pd.DataFrame(svm_model_params)
results_path = '../data/svm_svc_results_hj.csv'
svm_pd.to_csv(results_path, sep=',')

# svm_pd = pd.read_csv(results_path, sep=',', index_col=0)